# Data Preparation
Prepare data from GIS output for optimization model. Requires data by PPIC region (Kern County, Southeast, or Southwest are within analysis scope).

### Inputs:
* Shapefiles of ag fields with crop type ('Crop2014'), area in acres ('Acres'), field identifier ('FID_*County*'). 
    * Expected in `../spatial/` with County name ('Fresno', 'Tulare', 'Kings', 'Kern') appended to `'_fields'` (e.g, `'Fresno_fields.shp`').
* csv of conservation factor scores ('FID_*County*', 'TNC', 'Krat', 'Kitfox', 'Corridors', 'CPAD'). 
    * Expected in `../data/` with County name ('Fresno', 'Tulare', 'Kings', 'Kern') appended to `'_score'` (e.g, `'Fresno_score.csv'`).
* PPIC region shapefile with region code ('PPIC_Region': 'KR', 'NE', 'NW', 'SE', 'SW')
    * Expected in `../spatial/PPIC_Region.shp`
* csv to crosswalk LandIQ crop types with PPIC crop types
    * Expected in `../data/crop_x.csv`
* csv of rotation-adjusted price per acre ('AdjustedPrice'). Note these data were developed for Kern county and other counties are assumed to be equivalent.
    * Expected in`../data/crop_price.csv`
* csv of crop water consumption per acre ('*County*_AW').
    * Expected in`../data/crop_water.csv`

### Output:
* shapefile with per field ag data and conservation factor scores for entire analysis area.

### Process Overview:
1. Spatial join the county ag field shapefiles with PPIC region shapefile
2. For each region in the PPIC regions ('KR', 'SW', 'SE'):
    1. Lookup PPIC crop code from crop_x table
    2. Calculate price per acre
    3. Normalize conservation factors not already on a 1 - 100 scale
    4. Calculate area-weighted conservation production per field

## Inputs

Confirm the inputs

In [1]:
# Inputs
counties = ['Fresno', 'Tulare', 'Kings', 'Kern']
regions = ['KR', 'SE', 'SW']
ppic = r'../spatial/PPIC_Region.shp'
crop_x_table = r'../data/crop_x.csv'
crop_price_table = r'../data/crop_price.csv'
crop_water_table = r'../data/crop_water.csv'


# Path to output, use as input to future notebooks
out_shp = r'../outputs/fields_shape.shp'

## Processing

In [2]:
# Import statements
import pandas as pd
import numpy as np
import geopandas as gd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sns.set_style("white")

In [4]:
plt.rcParams["figure.figsize"] = (12,5)

## Read in Data

### Read in Conservation Factor Data

In [5]:
# Read in conservation factor scores, create new County column to identify county, 
# standardize all FID columns to 'FID', and combine
field_dfs = []
for county in counties:
    df = pd.read_csv('../data/' + county + '_score.csv')
    df.rename(columns={f'FID_{county}_fields': 'FID'}, inplace=True)
    df['County'] = county
    field_dfs.append(df)
df = pd.concat(field_dfs, sort=False)
df.head()

,FID,TNC,Krat,Kitfox,Corridors,CPAD,County
0,0,0.0,0.0,0.0,0.0,0.0,Fresno
1,1,0.0,0.0,0.0,0.0,0.0,Fresno
2,2,0.0,0.0,0.0,0.0,0.0,Fresno
3,3,0.0,0.0,0.0,0.0,0.0,Fresno
4,4,0.0,0.0,0.0,0.0,0.0,Fresno


### Read in Field Spatial Data

In [6]:
# Read in field shapefiles, drop extraneous columns, create new County column to identify county, 
# standardize all FID columns to 'FID', and combine
shp_list = []

for county in counties:
    shp = gd.read_file(f'../spatial/{county}_fields.shp')
    shp = shp[['Crop2014', 'Acres', f'FID_{county}', 'geometry']]
    shp.rename(columns={f'FID_{county}': 'FID'}, inplace=True)
    shp['County'] = county
    shp_list.append(shp)
    
fields = gd.GeoDataFrame(pd.concat(shp_list, ignore_index=True, sort=False))
fields.crs = {'init': 'epsg:6414'}
fields = fields.to_crs(epsg=3857)

In [7]:
fields.crs

{'init': 'epsg:3857', 'no_defs': True}

In [8]:
fields.head()

,Crop2014,Acres,FID,geometry,County
0,Grapes,43.474109,0,POLYGON Z ((-13355020.09666443 4407864.0678736...,Fresno
1,Grapes,93.723265,1,POLYGON Z ((-13331506.86330624 4374173.2907541...,Fresno
2,Peaches/Nectarines,6.409884,2,POLYGON Z ((-13304207.15885581 4380620.8643907...,Fresno
3,Grapes,17.461519,3,POLYGON Z ((-13331417.48071155 4381217.4603662...,Fresno
4,Grapes,9.365854,4,POLYGON Z ((-13339076.40436587 4383454.6129433...,Fresno


### Read in PPIC Spatial Data 

In [9]:
ppic_regions = gd.read_file(ppic)
ppic_regions = ppic_regions.to_crs(epsg=3857)
ppic_regions

,OBJECTID,PPIC_Regio,Shape_Leng,Shape_Area,geometry
0,1,KR,1.029789e+06,1.212854e+10,"POLYGON ((-13247038.307 4287718.055399997, -13..."
1,2,NE,8.712062e+05,1.149217e+10,"POLYGON ((-13501488.6241 4565348.2922, -135001..."
2,3,NW,7.630251e+05,4.508380e+09,"POLYGON ((-13502420.0527 4535907.504600001, -1..."
3,4,SE,1.069165e+06,9.642920e+09,"POLYGON ((-13325232.1049 4434083.094999996, -1..."
4,5,SW,8.410650e+05,1.094927e+10,"POLYGON ((-13371112.014 4413205.373400005, -13..."


### Read in Crop Crosswalk to PPIC table

In [10]:
crop_x = pd.read_csv(crop_x_table)
crop_x.head() 

,Crop,Crop_PPIC
0,AlfalfaandAlfalfaMixtures,alfalfa-pasture
1,Almonds,trees-vines
2,Apples,veg-fruits
3,Avocados,trees-vines
4,Beans(Dry),veg-fruits


### Read in Crop Info Table

In [11]:
crop_price = pd.read_csv(crop_price_table)
crop_price.head()

,Crop,Price,Rotations,AdjustedPrice
0,AlfalfaandAlfalfaMixtures,1113.653444,1.000000,1113.653444
1,Almonds,5995.531797,1.000000,5995.531797
2,Apples,1657.747573,1.000000,1657.747573
3,Avocados,13099.103230,1.000000,13099.103230
4,Beans(Dry),1060.225352,1.935484,2052.049069


In [12]:
crop_water = pd.read_csv(crop_water_table)
crop_water.head()

,Crop,Kern_AW,Fresno_AW,Kings_AW,Tulare_AW
0,AlfalfaandAlfalfaMixtures,5.08,4.589778,4.945137,5.125525
1,Almonds,4.54,3.521685,3.884410,3.889192
2,Apples,3.63,3.795877,3.985478,3.767993
3,Avocados,3.53,2.773072,3.172853,3.117050
4,Beans(Dry),3.26,2.137108,2.784316,3.071122


## Set Up Analysis

### Identify PPIC region

In [13]:
fields = gd.sjoin(fields, ppic_regions)

In [14]:
fields.groupby('PPIC_Regio')['County'].value_counts()

PPIC_Regio  County
KR          Kern      18651
            Tulare      707
            Kings         2
NW          Fresno     2381
SE          Tulare    29679
            Fresno    20113
            Kings      1487
            Kern         20
SW          Fresno    11178
            Kings      7100
            Kern         43
            Tulare       18
Name: County, dtype: int64

In [15]:
fields = fields[['Crop2014', 'Acres', 'FID', 'geometry', 'County', 'PPIC_Regio']]

In [16]:
fields.head()

,Crop2014,Acres,FID,geometry,County,PPIC_Regio
0,Grapes,43.474109,0,POLYGON Z ((-13355020.09666443 4407864.0678736...,Fresno,SE
1,Grapes,93.723265,1,POLYGON Z ((-13331506.86330624 4374173.2907541...,Fresno,SE
2,Peaches/Nectarines,6.409884,2,POLYGON Z ((-13304207.15885581 4380620.8643907...,Fresno,SE
3,Grapes,17.461519,3,POLYGON Z ((-13331417.48071155 4381217.4603662...,Fresno,SE
4,Grapes,9.365854,4,POLYGON Z ((-13339076.40436587 4383454.6129433...,Fresno,SE


### Cross-walk crop type to PPIC crop type

In [17]:
# Set up a list to track the columns that will be aggregated/optimized for ecosystem service benefits
eco_service_cols = np.array([])

In [18]:
fields = fields.join(crop_x.set_index('Crop'), on='Crop2014')

In [19]:
fields.head()

,Crop2014,Acres,FID,geometry,County,PPIC_Regio,Crop_PPIC
0,Grapes,43.474109,0,POLYGON Z ((-13355020.09666443 4407864.0678736...,Fresno,SE,trees-vines
1,Grapes,93.723265,1,POLYGON Z ((-13331506.86330624 4374173.2907541...,Fresno,SE,trees-vines
2,Peaches/Nectarines,6.409884,2,POLYGON Z ((-13304207.15885581 4380620.8643907...,Fresno,SE,trees-vines
3,Grapes,17.461519,3,POLYGON Z ((-13331417.48071155 4381217.4603662...,Fresno,SE,trees-vines
4,Grapes,9.365854,4,POLYGON Z ((-13339076.40436587 4383454.6129433...,Fresno,SE,trees-vines


In [20]:
#fields.to_file('../outputs/temp_fields.shp')

In [21]:
#fields = gd.read_file('../outputs/temp_fields.shp')

### Calculate Price per Acre

In [22]:
fields = fields.join(crop_price.set_index('Crop'), on='Crop2014')

#### Calculate FieldCropPrice column
Will be used to sort crop types within each PPIC crop type to select lowest revenue crops first when fallowing fields

In [23]:
fields['FieldCropPrice'] = round(fields['AdjustedPrice'] * fields['Acres'],2)

### Calculate Water Consumption

In [24]:
for county in counties:
    for crop in crop_water['Crop']:
        fields.loc[(fields['County']==county) & (fields['Crop2014']==crop), 'WaterConsumption'] = crop_water.loc[(crop_water['Crop']==crop), f'{county}_AW'].item()
        

In [25]:
fields.head()

,Crop2014,Acres,FID,geometry,County,PPIC_Regio,Crop_PPIC,Price,Rotations,AdjustedPrice,FieldCropPrice,WaterConsumption
0,Grapes,43.474109,0,POLYGON Z ((-13355020.09666443 4407864.0678736...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,688354.81,2.444929
1,Grapes,93.723265,1,POLYGON Z ((-13331506.86330624 4374173.2907541...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,1483983.52,2.444929
2,Peaches/Nectarines,6.409884,2,POLYGON Z ((-13304207.15885581 4380620.8643907...,Fresno,SE,trees-vines,15869.78488,1.0,15869.78488,101723.48,3.795877
3,Grapes,17.461519,3,POLYGON Z ((-13331417.48071155 4381217.4603662...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,276479.98,2.444929
4,Grapes,9.365854,4,POLYGON Z ((-13339076.40436587 4383454.6129433...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,148295.87,2.444929


####  Calculate SavedWater column

In [26]:
fields['SavedWater'] = round(fields['WaterConsumption'] * fields['Acres'], 2)

In [27]:
fields.head()

,Crop2014,Acres,FID,geometry,County,PPIC_Regio,Crop_PPIC,Price,Rotations,AdjustedPrice,FieldCropPrice,WaterConsumption,SavedWater
0,Grapes,43.474109,0,POLYGON Z ((-13355020.09666443 4407864.0678736...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,688354.81,2.444929,106.29
1,Grapes,93.723265,1,POLYGON Z ((-13331506.86330624 4374173.2907541...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,1483983.52,2.444929,229.15
2,Peaches/Nectarines,6.409884,2,POLYGON Z ((-13304207.15885581 4380620.8643907...,Fresno,SE,trees-vines,15869.78488,1.0,15869.78488,101723.48,3.795877,24.33
3,Grapes,17.461519,3,POLYGON Z ((-13331417.48071155 4381217.4603662...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,276479.98,2.444929,42.69
4,Grapes,9.365854,4,POLYGON Z ((-13339076.40436587 4383454.6129433...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,148295.87,2.444929,22.90


#### Normalize SavedWater

In [28]:
norm_columns = ['SavedWater']
for column in norm_columns:
    new_col = str(column + '_Norm')
    fields[new_col] = fields[column] / fields[column].max() * 100
    eco_service_cols = np.append(eco_service_cols, column)
fields.head()

,Crop2014,Acres,FID,geometry,County,PPIC_Regio,Crop_PPIC,Price,Rotations,AdjustedPrice,FieldCropPrice,WaterConsumption,SavedWater,SavedWater_Norm
0,Grapes,43.474109,0,POLYGON Z ((-13355020.09666443 4407864.0678736...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,688354.81,2.444929,106.29,5.037369
1,Grapes,93.723265,1,POLYGON Z ((-13331506.86330624 4374173.2907541...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,1483983.52,2.444929,229.15,10.860035
2,Peaches/Nectarines,6.409884,2,POLYGON Z ((-13304207.15885581 4380620.8643907...,Fresno,SE,trees-vines,15869.78488,1.0,15869.78488,101723.48,3.795877,24.33,1.153064
3,Grapes,17.461519,3,POLYGON Z ((-13331417.48071155 4381217.4603662...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,276479.98,2.444929,42.69,2.023194
4,Grapes,9.365854,4,POLYGON Z ((-13339076.40436587 4383454.6129433...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,148295.87,2.444929,22.90,1.085293


### Join Conservation Factor Table

In [29]:
fields = fields.join(df.set_index(['County', 'FID']), on=['County', 'FID'])

In [30]:
eco_service_cols = np.append(eco_service_cols, df.columns.values[1:-1]) #Exclude FID and County
eco_service_cols

array(['SavedWater', 'TNC', 'Krat', 'Kitfox', 'Corridors', 'CPAD'],
      dtype=object)

#### Calculate area-weighted ecosystem service values

In [31]:
for eco_service in eco_service_cols[1:]: # exclude Saved Water
    new_col = f'{eco_service}_Total'
    fields[new_col] = fields[eco_service] / 100 * fields['Acres']

In [32]:
fields.head()

,Crop2014,Acres,FID,geometry,County,PPIC_Regio,Crop_PPIC,Price,Rotations,AdjustedPrice,...,TNC,Krat,Kitfox,Corridors,CPAD,TNC_Total,Krat_Total,Kitfox_Total,Corridors_Total,CPAD_Total
0,Grapes,43.474109,0,POLYGON Z ((-13355020.09666443 4407864.0678736...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Grapes,93.723265,1,POLYGON Z ((-13331506.86330624 4374173.2907541...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Peaches/Nectarines,6.409884,2,POLYGON Z ((-13304207.15885581 4380620.8643907...,Fresno,SE,trees-vines,15869.78488,1.0,15869.78488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Grapes,17.461519,3,POLYGON Z ((-13331417.48071155 4381217.4603662...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Grapes,9.365854,4,POLYGON Z ((-13339076.40436587 4383454.6129433...,Fresno,SE,trees-vines,15833.67278,1.0,15833.67278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
fields.to_file(out_shp)

In [34]:
fields.columns

Index(['Crop2014', 'Acres', 'FID', 'geometry', 'County', 'PPIC_Regio',
       'Crop_PPIC', 'Price', 'Rotations', 'AdjustedPrice', 'FieldCropPrice',
       'WaterConsumption', 'SavedWater', 'SavedWater_Norm', 'TNC', 'Krat',
       'Kitfox', 'Corridors', 'CPAD', 'TNC_Total', 'Krat_Total',
       'Kitfox_Total', 'Corridors_Total', 'CPAD_Total'],
      dtype='object')

## Explore results 

#### Compare inputs to PPIC report
|Region|Land <br>(1,000s acres) <br>|Applied Water<br>(1,000s of acre-ft)<br>|Revenue <br>(2010 $, millions)<br>|
|:----:|:-------------:|:-----------------:|:----------------:|
|KR    |827           |2,958              |3,948|
|SE    |1,134         |3,662              |5,930|
|SW    |1,112         |3,177              |3,917|

In [35]:
total_revenue = fields['FieldCropPrice'].sum()
print (f'${total_revenue:,.2f} in annual revenue')

$17,975,027,885.17 in annual revenue


In [36]:
cropland = fields['Acres'].sum()
print(f'{cropland:,.0f} acres of cropland')

3,280,532 acres of cropland


In [37]:
applied_water = fields['SavedWater'].sum()
applied_water_s = (f'{applied_water:,.0f} acre-feet of applied water')

This example {{applied_water_s}} should work

In [38]:
pd.DataFrame({'Region': [region_code], 'Land':[f'{cropland:,.0f}'], 'Water':[f'{applied_water:,.0f}'], 'Revenue':[f'${total_revenue:,.0f}']})

NameError: name 'region_code' is not defined